### Import libraries and define node class

In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import ipywidgets as widgets
import random
import statistics

class ZollmanNode:
    def __init__(self, beliefProb, pEH=None):
        self.beliefProb = beliefProb
        self.pEH = pEH

In [2]:
# Create a random graph
globalVar = 10
graph = nx.erdos_renyi_graph(10, 0.3)


# Define interactive widgets for parameters
parameter_widget = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.1, description='Parameter:')


# Define the function for one timestep
def testing(graph, parameter):
    # Update the graph based on the parameter
    # This is a placeholder function, you should replace it with your own logic
    print(graph, parameter)
    pass

# Define a function to update the graph based on the parameter value
def update_graph(p1, parameter):
    testing(graph, parameter)
    nx.draw(graph, with_labels=True)
    plt.show()

# Create a function to start the simulation
def start_simulation():
    widgets.interact_manual(update_graph, p1=10, parameter=parameter_widget, manual_name="Custom Button Label")

# Display the widgets
start_simulation()


interactive(children=(IntSlider(value=10, description='p1', max=30, min=-10), FloatSlider(value=0.5, descripti…

### Define Function to Display Graph

### Define graph initialization function

In [8]:
def initialize_graph(graphType: str, numNodes: int = 10):
    graph = nx.Graph()
    if graphType == 'circle':
        for index in range(numNodes):
            initial_data = {'group': index % 4, 'beliefProb': random.random(), 'pEH': None}
            graph.add_node(f"scientist-{index}", **initial_data)
            if index > 0:
                graph.add_edge(f"scientist-{index - 1}", f"scientist-{index}")
        graph.add_edge(f"scientist-{numNodes - 1}", "scientist-0")
    elif graphType == 'wheel':
        for index in range(numNodes - 1):
            initial_data = {'group': index % 4, 'beliefProb': random.random(), 'pEH': None}
            graph.add_node(f"scientist-{index}", **initial_data)
            if index > 0:
                graph.add_edge(f"scientist-{index - 1}", f"scientist-{index}")
        graph.add_edge(f"scientist-{numNodes - 2}", "scientist-0")
        graph.add_node(f"scientist-{numNodes - 1}", group=1, beliefProb=random.random(), pEH=None)
        for node in graph.nodes:
            if node != f"scientist-{numNodes - 1}":
                graph.add_edge(node, f"scientist-{numNodes - 1}")
    else:
        for index in range(numNodes):
            initial_data = {'group': index % 4, 'beliefProb': random.random(), 'pEH': None}
            graph.add_node(f"scientist-{index}", **initial_data)
        for i in range(numNodes):
            for j in range(i + 1, numNodes):
                if i != j:
                    graph.add_edge(f"scientist-{i}", f"scientist-{j}")
    return graph

### Define timestep function and helper functions

In [67]:
def run_experiment(world):
    return random.uniform(0.3, 0.8) if world == 'higher' else random.uniform(0.2, 0.7)

def calculate_posterior(pH, pEH):
    pE = pEH * pH + (1 - pEH) * (1 - pH)
    return (pEH * pH) / pE

def timestep(graph):
    nodes = graph.nodes(data=True)
    for node, data in nodes:
        data['pEH'] = run_experiment('higher') if data['beliefProb'] > 0.5 else None
        data['beliefProb'] = calculate_posterior(data['beliefProb'], data['pEH']) if data['pEH'] is not None else data['beliefProb']

    posteriors = []
    for node, data in nodes:
        neighbors = graph.neighbors(node)
        for neighbor in neighbors:
            neighbor_data = graph.nodes[neighbor]
            if neighbor_data['pEH'] is not None:
                data['beliefProb'] = calculate_posterior(data['beliefProb'], neighbor_data['pEH'])
        data['group'] = sum(data['beliefProb'] > x for x in [0, 0.4, 0.6, 1])
        posteriors.append(data['beliefProb'])
    return posteriors

### Define Function to Run Simulation

In [71]:
def run_simulation(graph, num_timesteps=1):
    median_posteriors = []
    for _ in range(num_timesteps):
        posteriors = timestep(graph)
        median_posterior = statistics.median(posteriors)
        median_posteriors.append(median_posterior if median_posterior is not None else 0)
    return median_posteriors

### Run Simulation and Display Results

In [91]:
# Define Parameters
numTimesteps = 100 # or do not need this? Just run at a pace
newGraph = nx.Graph()
numNodes_label = widgets.Label("Number of Nodes:")
numNodes = widgets.BoundedIntText(value=10, min=2, max=25, step=1)
graphType = widgets.Select(options=['circle', 'wheel', 'complete'], value='circle', description='Graph Type:')
custom_button = widgets.Button(description="Initialize Graph")
# result = widgets.interact_manual(initialize_graph, graphType=graphType, numNodes=numNodes, description="Initialize Graph")
def onClick(_):
    graph2 = initialize_graph(numNodes=numNodes.value, graphType=graphType.value)
    newGraph.clear()
    newGraph.add_nodes_from(graph2.nodes)
    newGraph.add_edges_from(graph2.edges)
custom_button.on_click(onClick)
# Group the interactive widgets together using HBox or VBox
widget_group = widgets.VBox([numNodes_label, numNodes, graphType])

# Define timestep
timestepLabel = widgets.Label("Number of Timesteps:")
numTimesteps = widgets.BoundedIntText(value=100, min=1, max=10000, step=1)
timestepGroup = widgets.VBox([timestepLabel, numTimesteps])
start_button = widgets.Button(description="Run Simulation")

def onStart(_):
    if not newGraph.number_of_nodes:
        ValueError("No Graph initialized")
    print(numTimesteps.value)
    print(newGraph)
    run_simulation(graph=newGraph, num_timesteps=numTimesteps.value)
start_button.on_click(onStart)

# Display the custom button and the interactive widgets
display(widget_group, custom_button, timestepGroup, start_button)

# Buttons: initialize graph, run X timesteps

Button(description='Initialize Graph', style=ButtonStyle())

Button(description='Run Simulation', style=ButtonStyle())

100
Graph with 10 nodes and 10 edges


KeyError: 'beliefProb'

In [92]:
# Example usage:
graph = initialize_graph('circle', numNodes=10)
run_simulation(graph, num_timesteps=5)

[0.9146476416183658,
 0.9458599064883797,
 0.9888430715832884,
 0.9910226153378695,
 0.9866292644583958]